In [43]:
pip install boto3

Note: you may need to restart the kernel to use updated packages.


In [44]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
import os
import boto3

In [3]:
os.getcwd()

'c:\\Users\\nicol\\Desktop\\Kayakproject'

In [4]:
dfweather=pd.read_csv("Citieswithweather.csv")
dfhotels=pd.read_csv("Hotels.csv")

In [12]:
dfmerged=pd.merge(dfweather,dfhotels,on="Destination",suffixes=("_Destination","_Hotels")).drop(columns=["Unnamed: 0_Destination",'Unnamed: 0_Hotels',"bbox"])

In [13]:
dfmerged.head()

,Destination,Lat_Destination,Lon_Destination,AverageFeltTemperature,HoursOfSun,Percentageofchanceofrain,WeatherTypeScore,WeatherScore,DestinationID,name,url,Rating,description,Lat_Hotels,Lon_Hotels
0,Mont Saint Michel,48.635954,-1.51146,12.46,14.61,0.1,6.14,38.43,0,Le Relais Saint Michel,https://www.booking.com/hotel/fr/le-relais-sai...,"7,8","[""Vous pouvez bénéficier d'une réduction Geniu...",48.617587,-1.510396
1,Mont Saint Michel,48.635954,-1.51146,12.46,14.61,0.1,6.14,38.43,0,Mercure Mont Saint Michel,https://www.booking.com/hotel/fr/mont-saint-mi...,"8,2","[""Installé dans des espaces verts à seulement ...",48.614247,-1.510545
2,Mont Saint Michel,48.635954,-1.51146,12.46,14.61,0.1,6.14,38.43,0,Le Saint Aubert,https://www.booking.com/hotel/fr/hotel-saint-a...,"7,2","['Niché dans un écrin de verdure, à seulement ...",48.612938,-1.510105
3,Mont Saint Michel,48.635954,-1.51146,12.46,14.61,0.1,6.14,38.43,0,Hotel Gabriel,https://www.booking.com/hotel/fr/hotel-gabriel...,"8,0","[""Vous pouvez bénéficier d'une réduction Geniu...",48.615381,-1.510710
4,Mont Saint Michel,48.635954,-1.51146,12.46,14.61,0.1,6.14,38.43,0,Hotel De La Digue,https://www.booking.com/hotel/fr/de-la-digue.f...,"7,2","[""Cet établissement est à 1 minute à pied de l...",48.616882,-1.510918


In [14]:
dfmerged.to_csv("Combineddatabase.csv")

In [21]:
dfmerged['Rating']

0      7.8
1      8.2
2      7.2
3      8.0
4      7.2
      ... 
878    7.5
879    8.3
880    9.2
881    8.7
882    6.8
Name: Rating, Length: 883, dtype: float64

In [29]:
dfmerged['Rating']=dfmerged['Rating'].fillna(3)

In [22]:
dfmerged['Rating'] = dfmerged['Rating'].str.replace(',', '.').astype(float)


AttributeError: Can only use .str accessor with string values!

In [38]:
dfmerged['Ratingforgraph']=dfmerged["Rating"]**10

In [56]:

fig = px.scatter_mapbox(dfmerged, lat="Lat_Hotels", lon="Lon_Hotels", color="WeatherScore", size="Ratingforgraph", hover_name="name",hover_data={"Destination":True,"Rating":True,"WeatherScore":True,"Ratingforgraph":False,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat_Hotels":False,"Lon_Hotels":False,},zoom=4, mapbox_style="carto-positron", title="Map of 35 best destinations with their best hotels, ranked by best weather and hotel rating")
fig.show()

In [57]:

fig = px.scatter_mapbox(dfmerged.sort_values("Rating").tail(20), lat="Lat_Hotels", lon="Lon_Hotels", color="Rating", size="Ratingforgraph", hover_name="name",hover_data={"Destination":True,"Rating":True,"WeatherScore":True,"Ratingforgraph":False,"AverageFeltTemperature":True,"HoursOfSun":True,"Percentageofchanceofrain":True,"WeatherTypeScore":True,"Lat_Hotels":False,"Lon_Hotels":False,},zoom=4, mapbox_style="carto-positron", title="Map of 20 best hotels, ranked by hotel rating")
fig.show()

In [65]:
dfkeys=pd.read_csv("keys.csv")

In [72]:
session = boto3.Session(aws_access_key_id=dfkeys["aws_access_key_id"][0], 
                        aws_secret_access_key=dfkeys["aws_secret_access_key"][0]+"/Hfn")

In [73]:
s3 = session.resource("s3")

In [74]:
bucket = s3.Bucket("kayakprojectnicolasjehda")

In [55]:
bucket.upload_file("Combineddatabase.csv", "Combineddatabase.csv")